In [1]:
import os
print(f"현재 PID: {os.getpid()}")

현재 PID: 4120241


In [ ]:
import torch
from my_model import ResNet18_CBAM_MGA

model = ResNet18_CBAM_MGA()
print(f"Total params: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# ResNet18 + 데이터 증강 (CenterCrop 224 / RandomHorizontalFlip / RandomRotation / GaussianBlur)

import os, numpy as np, torch, gc, csv
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from glob import glob
from tqdm import tqdm
import torchvision.transforms as transforms
from torchvision.models import resnet18
from datetime import datetime
from PIL import Image

# 디바이스 설정
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 경로 및 하이퍼파라미터 설정
slice_root = "/data1/lidc-idri/slices"
batch_size = 16
num_epochs = 100
learning_rate = 1e-4
save_path = os.path.join("pth", "resnet18_best_aug2.pth")
os.makedirs("logs", exist_ok=True)
log_path = os.path.join("logs", "best_model_log2.csv")

# -------------------- Transform --------------------
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.GaussianBlur(kernel_size=(5, 5), sigma=(0.1, 2.0)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# 라벨 추출 함수
def extract_label_from_filename(fname):
    try:
        score = int(fname.split("_")[-1].replace(".npy", ""))
        return None if score == 3 else int(score >= 4)
    except:
        return None

# 데이터셋 정의
class CTDataset(Dataset):
    def __init__(self, paths, labels, transform=None):
        self.paths = paths
        self.labels = labels
        self.transform = transform

    def __getitem__(self, idx):
        file_path = self.paths[idx]
        label = self.labels[idx]
        img = np.load(file_path)
        img = np.clip(img, -1000, 400)
        img = (img + 1000) / 1400.
        img = np.expand_dims(img, axis=-1)
        if self.transform:
            img = self.transform(img)
        else:
            img = torch.tensor(img.transpose(2, 0, 1), dtype=torch.float32)
        return img, torch.tensor(label).long()

    def __len__(self):
        return len(self.paths)

# CSV 기록 함수
def log_best_experiment_result(log_path, model_name, exp_tag, epoch, val_acc, test_acc, auc, precision, recall, specificity, f1, notes=""):
    log_exists = os.path.exists(log_path)
    with open(log_path, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        if not log_exists:
            writer.writerow(["timestamp", "model", "exp_tag", "epoch", "val_acc", "test_acc", "auc", "precision", "recall", "specificity", "f1", "notes"])
        writer.writerow([
            datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            model_name,
            exp_tag,
            epoch,
            f"{val_acc:.4f}",
            f"{test_acc:.4f}",
            f"{auc:.4f}",
            f"{precision:.4f}",
            f"{recall:.4f}",
            f"{specificity:.4f}",
            f"{f1:.4f}",
            notes
        ])

# 학습 루프
def run():
    all_files = glob(os.path.join(slice_root, "LIDC-IDRI-*", "*.npy"))
    file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
    file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]
    files, labels = zip(*file_label_pairs)
    train_files, temp_files, train_labels, temp_labels = train_test_split(files, labels, test_size=0.3, random_state=42)
    val_files, test_files, val_labels, test_labels = train_test_split(temp_files, temp_labels, test_size=0.5, random_state=42)

    train_loader = DataLoader(CTDataset(train_files, train_labels, transform=train_transform), batch_size=batch_size, num_workers=4, pin_memory=True)
    val_loader = DataLoader(CTDataset(val_files, val_labels, transform=val_transform), batch_size=batch_size)
    test_loader = DataLoader(CTDataset(test_files, test_labels, transform=val_transform), batch_size=batch_size)

    model = resnet18(weights=None)
    model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
    model.fc = nn.Linear(model.fc.in_features, 2)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    best_acc = 0.0
    best_epoch = 0

    for epoch in range(num_epochs):
        model.train()
        epoch_loss, correct, total = 0, 0, 0
        for images, labels in tqdm(train_loader, desc=f"[Epoch {epoch+1}]"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            _, preds = outputs.max(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            epoch_loss += loss.item()
        print(f"Train Acc: {(correct/total)*100:.4f}, Loss: {epoch_loss/len(train_loader):.4f}")

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                _, preds = outputs.max(1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        val_acc = correct / total
        print(f"Val Acc: {val_acc:.4f}")

        if val_acc > best_acc:
            best_acc = val_acc
            best_epoch = epoch + 1
            torch.save(model.state_dict(), save_path)
            print("✅ Saved best model!")

    # 테스트
    print("\n📊 Test Evaluation:")
    model.load_state_dict(torch.load(save_path))
    model.eval()
    y_true, y_pred, y_probs = [], [], []
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            probs = F.softmax(outputs, dim=1)[:, 1]
            preds = outputs.argmax(1)
            y_probs.extend(probs.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(labels.cpu().numpy())

    y_true, y_pred = np.array(y_true), np.array(y_pred)
    test_acc = (y_true == y_pred).mean()
    auc = roc_auc_score(y_true, y_probs)
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)
    specificity = tn / (tn + fp + 1e-8)
    f1 = 2 * precision * recall / (precision + recall + 1e-8)

    print(f"✅ Test Accuracy: {test_acc * 100:.2f}%")
    print(f"AUC: {auc:.4f}")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, Specificity: {specificity:.4f}, F1: {f1:.4f}")
    print("Confusion Matrix:")
    print(cm)

    log_best_experiment_result(
        log_path=log_path,
        model_name="ResNet18",
        exp_tag="aug2_flip_rot_erase",
        epoch=best_epoch,
        val_acc=best_acc,
        test_acc=test_acc,
        auc=auc,
        precision=precision,
        recall=recall,
        specificity=specificity,
        f1=f1,
        notes="CenResizeterCrop / RandomHorizontalFlip / RandomRotation / RandomErasing"
    )

if __name__ == "__main__":
    run()

Using device: cuda:1


[Epoch 1]: 100%|██████████| 234/234 [00:06<00:00, 33.46it/s]


Train Acc: 64.9518, Loss: 0.6524
Val Acc: 0.6887
✅ Saved best model!


[Epoch 2]: 100%|██████████| 234/234 [00:08<00:00, 28.22it/s]


Train Acc: 65.6752, Loss: 0.6412
Val Acc: 0.6887


...

Train Acc: 99.0354, Loss: 0.0311
Val Acc: 0.6625


[Epoch 99]: 100%|██████████| 234/234 [00:08<00:00, 28.15it/s]


Train Acc: 98.8210, Loss: 0.0395
Val Acc: 0.6687


[Epoch 100]: 100%|██████████| 234/234 [00:07<00:00, 32.24it/s]


Train Acc: 98.5263, Loss: 0.0395
Val Acc: 0.6713

📊 Test Evaluation:
✅ Test Accuracy: 65.62%
AUC: 0.4791
Precision: 0.6578, Recall: 0.9924, Specificity: 0.0145, F1: 0.7912
Confusion Matrix:
[[  4 271]
 [  4 521]]


In [ ]:
# ResNet18 + 데이터 증강 (Resize((224, 224) / RandomHorizontalFlip / RandomRotation / RandomErasing)

import os, numpy as np, torch, gc, csv
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from glob import glob
from tqdm import tqdm
import torchvision.transforms as transforms
from torchvision.models import resnet18
from datetime import datetime
from PIL import Image

# 디바이스 설정
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 경로 및 하이퍼파라미터 설정
slice_root = "/data1/lidc-idri/slices"
batch_size = 16
num_epochs = 100
learning_rate = 1e-4
save_path = os.path.join("pth", "resnet18_best_aug1.pth")
os.makedirs("logs", exist_ok=True)
log_path = os.path.join("logs", "best_model_log1.csv")

# Transform 설정
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.1), ratio=(0.3, 3.3), value=0)
])

val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# 라벨 추출 함수
def extract_label_from_filename(fname):
    try:
        score = int(fname.split("_")[-1].replace(".npy", ""))
        return None if score == 3 else int(score >= 4)
    except:
        return None

# 데이터셋 정의
class CTDataset(Dataset):
    def __init__(self, paths, labels, transform=None):
        self.paths = paths
        self.labels = labels
        self.transform = transform

    def __getitem__(self, idx):
        file_path = self.paths[idx]
        label = self.labels[idx]
        img = np.load(file_path)
        img = np.clip(img, -1000, 400)
        img = (img + 1000) / 1400.
        img = np.expand_dims(img, axis=-1)
        if self.transform:
            img = self.transform(img)
        else:
            img = torch.tensor(img.transpose(2, 0, 1), dtype=torch.float32)
        return img, torch.tensor(label).long()

    def __len__(self):
        return len(self.paths)

# CSV 기록 함수
def log_best_experiment_result(log_path, model_name, exp_tag, epoch, val_acc, test_acc, auc, precision, recall, specificity, f1, notes=""):
    log_exists = os.path.exists(log_path)
    with open(log_path, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        if not log_exists:
            writer.writerow(["timestamp", "model", "exp_tag", "epoch", "val_acc", "test_acc", "auc", "precision", "recall", "specificity", "f1", "notes"])
        writer.writerow([
            datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            model_name,
            exp_tag,
            epoch,
            f"{val_acc:.4f}",
            f"{test_acc:.4f}",
            f"{auc:.4f}",
            f"{precision:.4f}",
            f"{recall:.4f}",
            f"{specificity:.4f}",
            f"{f1:.4f}",
            notes
        ])

# 학습 루프
def run():
    all_files = glob(os.path.join(slice_root, "LIDC-IDRI-*", "*.npy"))
    file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
    file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]
    files, labels = zip(*file_label_pairs)
    train_files, temp_files, train_labels, temp_labels = train_test_split(files, labels, test_size=0.3, random_state=42)
    val_files, test_files, val_labels, test_labels = train_test_split(temp_files, temp_labels, test_size=0.5, random_state=42)

    train_loader = DataLoader(CTDataset(train_files, train_labels, transform=train_transform), batch_size=batch_size, num_workers=4, pin_memory=True)
    val_loader = DataLoader(CTDataset(val_files, val_labels, transform=val_transform), batch_size=batch_size)
    test_loader = DataLoader(CTDataset(test_files, test_labels, transform=val_transform), batch_size=batch_size)

    model = resnet18(weights=None)
    model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
    model.fc = nn.Linear(model.fc.in_features, 2)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    best_acc = 0.0
    best_epoch = 0

    for epoch in range(num_epochs):
        model.train()
        epoch_loss, correct, total = 0, 0, 0
        for images, labels in tqdm(train_loader, desc=f"[Epoch {epoch+1}]"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            _, preds = outputs.max(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            epoch_loss += loss.item()
        print(f"Train Acc: {(correct/total)*100:.4f}, Loss: {epoch_loss/len(train_loader):.4f}")

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                _, preds = outputs.max(1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        val_acc = correct / total
        print(f"Val Acc: {val_acc:.4f}")

        if val_acc > best_acc:
            best_acc = val_acc
            best_epoch = epoch + 1
            torch.save(model.state_dict(), save_path)
            print("✅ Saved best model!")

    # 테스트
    print("\n📊 Test Evaluation:")
    model.load_state_dict(torch.load(save_path))
    model.eval()
    y_true, y_pred, y_probs = [], [], []
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            probs = F.softmax(outputs, dim=1)[:, 1]
            preds = outputs.argmax(1)
            y_probs.extend(probs.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(labels.cpu().numpy())

    y_true, y_pred = np.array(y_true), np.array(y_pred)
    test_acc = (y_true == y_pred).mean()
    auc = roc_auc_score(y_true, y_probs)
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)
    specificity = tn / (tn + fp + 1e-8)
    f1 = 2 * precision * recall / (precision + recall + 1e-8)

    print(f"✅ Test Accuracy: {test_acc * 100:.2f}%")
    print(f"AUC: {auc:.4f}")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, Specificity: {specificity:.4f}, F1: {f1:.4f}")
    print("Confusion Matrix:")
    print(cm)

    log_best_experiment_result(
        log_path=log_path,
        model_name="ResNet18",
        exp_tag="aug1_flip_rot_erase",
        epoch=best_epoch,
        val_acc=best_acc,
        test_acc=test_acc,
        auc=auc,
        precision=precision,
        recall=recall,
        specificity=specificity,
        f1=f1,
        notes="CenResizeterCrop / RandomHorizontalFlip / RandomRotation / RandomErasing"
    )

if __name__ == "__main__":
    run()

Using device: cuda:1


[Epoch 1]: 100%|██████████| 234/234 [00:08<00:00, 28.98it/s]


Train Acc: 64.9518, Loss: 0.6354
Val Acc: 0.6900
✅ Saved best model!


[Epoch 2]: 100%|██████████| 234/234 [00:07<00:00, 30.43it/s]


Train Acc: 67.2562, Loss: 0.6118
Val Acc: 0.6925
✅ Saved best model!


...

Train Acc: 97.9904, Loss: 0.0535
Val Acc: 0.8825


[Epoch 99]: 100%|██████████| 234/234 [00:07<00:00, 32.01it/s]


Train Acc: 97.6420, Loss: 0.0602
Val Acc: 0.8912


[Epoch 100]: 100%|██████████| 234/234 [00:07<00:00, 32.67it/s]


Train Acc: 98.0975, Loss: 0.0559
Val Acc: 0.8850

📊 Test Evaluation:
✅ Test Accuracy: 88.00%
AUC: 0.9331
Precision: 0.9055, Recall: 0.9124, Specificity: 0.8182, F1: 0.9089
Confusion Matrix:
[[225  50]
 [ 46 479]]


In [ ]:
# ResNet18 + Resize 224

import os, numpy as np, torch, gc
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from glob import glob
from tqdm import tqdm
import torchvision.transforms as transforms
from torchvision.models import resnet18
from PIL import Image

# 디바이스 설정
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 경로 및 하이퍼파라미터 설정
slice_root = "/data1/lidc-idri/slices"
bbox_csv_path = "/home/iujeong/lung_cancer/csv/allbb_noPoly.csv"
batch_size = 16
num_epochs = 100
learning_rate = 1e-4

# 최소한의 전처리 Transform
tf = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# 라벨 추출 함수
def extract_label_from_filename(fname):
    try:
        score = int(fname.split("_")[-1].replace(".npy", ""))
        return None if score == 3 else int(score >= 4)
    except:
        return None

# 데이터셋 정의
class CTDataset(Dataset):
    def __init__(self, paths, labels, transform=None):
        self.paths = paths
        self.labels = labels
        self.transform = transform

    def __getitem__(self, idx):
        file_path = self.paths[idx]
        label = self.labels[idx]
        img = np.load(file_path)
        img = np.clip(img, -1000, 400)
        img = (img + 1000) / 1400.
        img = np.expand_dims(img, axis=-1)
        img = self.transform(img) if self.transform else torch.tensor(img.transpose(2, 0, 1), dtype=torch.float32)
        return img, torch.tensor(label).long()

    def __len__(self):
        return len(self.paths)

# 학습 루프
def run():
    all_files = glob(os.path.join(slice_root, "LIDC-IDRI-*", "*.npy"))
    file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
    file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]
    files, labels = zip(*file_label_pairs)
    train_files, temp_files, train_labels, temp_labels = train_test_split(files, labels, test_size=0.3, random_state=42)
    val_files, test_files, val_labels, test_labels = train_test_split(temp_files, temp_labels, test_size=0.5, random_state=42)

    train_loader = DataLoader(CTDataset(train_files, train_labels, transform=tf), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(CTDataset(val_files, val_labels, transform=tf), batch_size=batch_size)
    test_loader = DataLoader(CTDataset(test_files, test_labels, transform=tf), batch_size=batch_size)

    model = resnet18(weights=None)
    model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
    model.fc = nn.Linear(model.fc.in_features, 2)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    best_acc = 0.0
    save_path = os.path.join(os.path.dirname(os.getcwd()), "pth", "resnet18_baseline.pth")

    for epoch in range(num_epochs):
        model.train()
        epoch_loss, correct, total = 0, 0, 0
        for images, labels in tqdm(train_loader, desc=f"[Epoch {epoch+1}]"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            _, preds = outputs.max(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            epoch_loss += loss.item()
        print(f"Train Acc: {(correct/total)*100:.4f}, Loss: {epoch_loss/len(train_loader):.4f}")

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                _, preds = outputs.max(1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        val_acc = correct / total
        print(f"Val Acc: {val_acc:.4f}")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), save_path)
            print("✅ Saved best model!")

    print("\n📊 Test Evaluation:")
    model.load_state_dict(torch.load(save_path))
    model.eval()
    y_true, y_pred, y_probs = [], [], []
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            probs = F.softmax(outputs, dim=1)[:, 1]
            preds = outputs.argmax(1)
            y_probs.extend(probs.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(labels.cpu().numpy())
    print(f"✅ Test Accuracy: {(np.array(y_pred) == np.array(y_true)).mean() * 100:.2f}%")
    print(classification_report(y_true, y_pred, digits=4))
    print(f"AUC: {roc_auc_score(y_true, y_probs):.4f}")
    print("Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))

if __name__ == "__main__":
    run()


Using device: cuda:1


[Epoch 1]: 100%|██████████| 234/234 [00:26<00:00,  8.77it/s]


Train Acc: 69.6141, Loss: 0.5848
Val Acc: 0.7412
✅ Saved best model!


[Epoch 2]: 100%|██████████| 234/234 [00:24<00:00,  9.36it/s]


Train Acc: 84.4855, Loss: 0.3648
Val Acc: 0.7925
✅ Saved best model!


...

Train Acc: 99.5713, Loss: 0.0128
Val Acc: 0.8812


[Epoch 99]: 100%|██████████| 234/234 [00:28<00:00,  8.10it/s]


Train Acc: 99.6785, Loss: 0.0073
Val Acc: 0.8838


[Epoch 100]: 100%|██████████| 234/234 [00:30<00:00,  7.79it/s]


Train Acc: 99.7856, Loss: 0.0031
Val Acc: 0.8838

📊 Test Evaluation:
✅ Test Accuracy: 88.75%
              precision    recall  f1-score   support

           0     0.8491    0.8182    0.8333       275
           1     0.9065    0.9238    0.9151       525

    accuracy                         0.8875       800
   macro avg     0.8778    0.8710    0.8742       800
weighted avg     0.8868    0.8875    0.8870       800

AUC: 0.9332
Confusion Matrix:
[[225  50]
 [ 40 485]]


In [ ]:
import os
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import torch.nn.functional as F
from tqdm import tqdm
from collections import defaultdict
import gc

# 하이퍼파라미터 설정
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)

slice_root = "/data1/lidc-idri/slices"
batch_size = 16
num_epoch = 1
learning_rate = 1e-4

# 레이블 추출
def labels_filename(fname):
    try:
        score = int(fname.split("_")[-1].replace(".npy", ""))
        return None  if score == 3 else int(score >= 4)
    
    except:
        return None
    
# 데이터셋 전처리
class LIDCDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)
    
    def __getitem__(self, index):
        file_path = self.file_paths[index]
        label = self.labels[index]

        img = np.load(file_path).astype(np.float32)
        img = np.clip(img, -1000, 400)
        img = (img + 1000) / 1400.0
        img = np.expand_dims(img, axis=0)
        img_tensor = torch.tensor(img)

        if self.transform:
            img_tensor = self.transform(img_tensor)

        return img_tensor, torch.tensor(label).float()


# 데이터 증강
augmentation_configs = {
    'baseline': transforms.Compose([
        transforms.ToPILImage(),
        transforms.CenterCrop(180),
        transforms.ToTensor()
    ]),

    'flip_rotate': transforms.Compose([
        transforms.ToPILImage(),
        transforms.CenterCrop(180),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ToTensor()
    ]),

    'blur': transforms.Compose([
        transforms.ToPILImage(),
        transforms.CenterCrop(180),
        transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
        transforms.ToTensor()
    ]),

    'total': transforms.Compose([
        transforms.ToPILImage(),
        transforms.CenterCrop(180),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
        transforms.ToTensor()
    ])
}

# 데이터 불러오기
def get_model(name):
    if name == "resnet18":
        model = models.resnet18(pretrained=True)
        model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        model.fc = nn.Linear(model.fc.in_features, 1)

    elif name == "resnet34":
        model = models.resnet34(pretrained=True)
        model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        model.fc = nn.Linear(model.fc.in_features, 1)

    elif name == "densenet121":
        model = models.densenet121(pretrained=True)
        model.features.conv0 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        model.classifier = nn.Linear(model.classifier.in_features, 1)

    elif name == "efficientnet_b0":
        model = models.efficientnet_b0(pretrained=True)
        model.features[0][0] = nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1, bias=False)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, 1)
    else:
        raise ValueError("Unknown model name")
    return model.to(device)


# 데이터 로더
all_files = glob(os.path.join(slice_root, "LIDC-IDRI-*", "*.npy"))

file_label_pairs = [(f, labels_filename(f)) for f in all_files]
file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]
files, labels = zip(*file_label_pairs)

train_files, temp_files, train_labels, temp_labels = train_test_split(files, labels, test_size=0.3, random_state=42)
val_files, test_files, val_labels, test_labels = train_test_split(temp_files, temp_labels, test_size=0.5, random_state=42)


# 모델 정의
results = defaultdict(dict)
model_names = ["resnet18", "resnet34", "densenet121", "efficientnet_b0"]

for model_name in model_names:
    for aug_name, transform in augmentation_configs.items():
        print(f"\n Running: {model_name} + {aug_name}")

        train_dataset = LIDCDataset(train_files, train_labels, transform)
        val_dataset = LIDCDataset(val_files, val_labels, transform)
        test_dataset = LIDCDataset(test_files, test_labels, transform)

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)
        test_loader = DataLoader(test_dataset, batch_size=batch_size)

        model = get_model(model_name)

# loss, optimizer 설정
        criterion = nn.BCEWithLogitsLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


# 반복문
        best_val_acc = 0.0

        # --- 저장 경로 및 변수 초기화 ---
        save_dir = os.path.join(os.path.dirname(os.getcwd()), "pth")
        os.makedirs(save_dir, exist_ok=True)
        best_val_acc = 0.0

        for epoch in range(num_epoch):
            model.train()

            correct = 0
            total = 0
            epoch_loss = 0

            for images, labels in train_loader:
                images = images.to(device)
                labels = labels.unsqueeze(1).to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                predicted = (torch.sigmoid(outputs) > 0.5).long()
                correct += (predicted == labels.long()).sum().item()
                total += labels.size(0)

            train_acc = correct / total
            print(f"[{model_name} + {aug_name}] Epoch: {epoch+1}/{num_epoch} Train Acc: {train_acc * 100:.4f}%")

            model.eval()

            val_correct = 0
            val_total = 0

            with torch.no_grad():
                for images, labels in val_loader:
                    images = images.to(device)
                    labels = labels.to(device)

                    outputs = model(images)

                    predicted = (torch.sigmoid(outputs) > 0.5).squeeze().long()
                    val_correct += (predicted == labels.long()).sum().item()
                    val_total += labels.size(0)

            val_acc = val_correct / val_total
            print(f"[{model_name} + {aug_name}] Epoch {epoch+1}/{num_epoch} Val Acc {val_acc * 100:.4f}%")

            if val_acc > best_val_acc:
                best_val_acc = val_acc
                torch.save(model.state_dict(), os.path.join(save_dir, f"best_aug_{model_name}_{aug_name}.pth"))


        model.load_state_dict(torch.load(os.path.join(save_dir, f"best_aug_{model_name}_{aug_name}.pth")))
        model.eval()

        y_true, y_pred, y_probs = [], [], []

        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)

                probs = torch.sigmoid(outputs).squeeze()
                preds = (probs > 0.5).long()

                y_true.extend(labels.cpu().numpy())
                y_pred.extend(preds.cpu().numpy())
                y_probs.extend(probs.cpu().numpy())

        acc = (np.array(y_true) == np.array(y_pred)).mean()
        auc = roc_auc_score(y_true, y_probs)
        cm = confusion_matrix(y_true, y_pred)
        results[model_name][aug_name] = {"acc": acc, "auc": auc, "cm": cm}
        print(f"✅ Test Acc: {acc:.4f}, AUC: {auc:.4f}")


In [ ]:
import os
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import torch.nn.functional as F
from tqdm import tqdm
import gc


# --- 설정 ---
SLICE_ROOT = "/data1/lidc-idri/slices"
BATCH_SIZE = 16
NUM_EPOCHS = 100
LR = 1e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- 라벨 추출 함수 ---
def extract_label_from_filename(filename):
    try:
        score = int(filename.split("_")[-1].replace(".npy", ""))
        if score == 3:
            return None
        return 1 if score >= 4 else 0
    except:
        return None

# --- 파일 리스트 구성 ---
all_files = glob(os.path.join(SLICE_ROOT, "LIDC-IDRI-*", "*.npy"))
file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]
files, labels = zip(*file_label_pairs)

# --- 3-way split (70% train / 15% val / 15% test) ---
train_files, temp_files, train_labels, temp_labels = train_test_split(
    files, labels, test_size=0.3, random_state=42
)
val_files, test_files, val_labels, test_labels = train_test_split(
    temp_files, temp_labels, test_size=0.5, random_state=42
)

# --- Transform 정의 ---
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.CenterCrop(180),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    transforms.ToTensor()
])

val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.CenterCrop(180),
    transforms.ToTensor()
])

# --- Dataset 정의 ---
class LIDCDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, index):
        file_path = self.file_paths[index]
        label = self.labels[index]

        img = np.load(file_path)
        img = np.clip(img, -1000, 400)
        img = ((img + 1000) / 1400.0 * 255).astype(np.uint8)  # uint8로 변환
        img = img.squeeze()  # shape: (H, W)

        if self.transform:
            img = self.transform(img)  # img: np.uint8 (H, W) → PIL → tensor(C, H, W)
        else:
            img = torch.tensor(img / 255.0).unsqueeze(0).float()

        return img, torch.tensor(label).float()

# --- DataLoader 정의 ---
train_dataset = LIDCDataset(train_files, train_labels, transform=train_transform)
val_dataset = LIDCDataset(val_files, val_labels, transform=val_transform)
test_dataset = LIDCDataset(test_files, test_labels, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# --- 모델 정의 ---
model = models.resnet18(pretrained=True)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.fc = nn.Linear(model.fc.in_features, 1)
model = model.to(DEVICE)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

# --- 저장 경로 및 변수 초기화 ---
save_dir = os.path.join(os.path.dirname(os.getcwd()), "pth")
os.makedirs(save_dir, exist_ok=True)
best_val_acc = 0.0

# --- 학습 루프 ---
for epoch in range(NUM_EPOCHS):
    model.train()

    epoch_loss = 0
    correct = 0
    total = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}"):
        images = images.to(DEVICE)
        labels = labels.unsqueeze(1).to(DEVICE)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        predicted = torch.sigmoid(outputs) >= 0.5
        total += labels.size(0)
        correct += (predicted == labels.long()).sum().item()

        epoch_loss += loss.item()
        train_acc = correct / total

    print(f"[Epoch {epoch+1}] Train Loss: {epoch_loss/len(train_loader):.4f} Accuracy : {train_acc * 100:.2f}%")

    gc.collect()
    torch.cuda.empty_cache()

    # --- Validation ---
    model.eval()
    correct = total = 0
    y_true = []
    y_pred = []

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)

            outputs = model(images)
            preds = (torch.sigmoid(outputs) > 0.5).squeeze().long()

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

            correct += (preds == labels.long()).sum().item()
            total += labels.size(0)

    val_acc = correct / total
    print(f"Validation Accuracy: {val_acc * 100:.2f}%")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), os.path.join(save_dir, "model_aug_resnet18.pth"))
        print("✅ Best model saved!")


# --- Test (최종 평가) ---
print("\n\U0001F4CA Test Set Evaluation (Best Model \uAE30\uc900):")
model.load_state_dict(torch.load(os.path.join(save_dir, "model_aug_resnet18.pth")))
model.eval()

correct = total = 0
y_true = []
y_pred = []
y_probs = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(DEVICE)
        labels = labels.unsqueeze(1).float().to(DEVICE)

        outputs = model(images)
        probs = torch.sigmoid(outputs).squeeze()
        preds = (probs > 0.5).long()

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())
        y_probs.extend(probs.cpu().numpy())

        correct += (preds == labels.long()).sum().item()
        total += labels.size(0)

test_acc = correct / total
print(f"\u2705 Test Accuracy: {test_acc * 100:.2f}%")
print(classification_report(y_true, y_pred, digits=4))

# AUC 출력
try:
    auc_score = roc_auc_score(y_true, y_probs)
    print(f"AUC: {auc_score:.4f}")
except ValueError:
    print("AUC \uacc4\uc0b0 \uc2e4\ud328: \uc591/\uc74c \ud074\ub798\uc2a4\uac00 \ubaa8\ub450 \uc788\uc5b4\uc57c \ud568.")

# Confusion Matrix 출력
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)


Epoch 1/100: 100%|██████████| 234/234 [00:28<00:00,  8.13it/s]


[Epoch 1] Train Loss: 0.6589 Accuracy : 62.67%
Validation Accuracy: 67.12%
✅ Best model saved!


Epoch 2/100: 100%|██████████| 234/234 [00:29<00:00,  7.81it/s]


[Epoch 2] Train Loss: 0.6071 Accuracy : 67.87%
Validation Accuracy: 66.00%


Epoch 3/100: 100%|██████████| 234/234 [00:25<00:00,  9.03it/s]


...

[Epoch 98] Train Loss: 0.0227 Accuracy : 99.28%
Validation Accuracy: 86.50%


Epoch 99/100: 100%|██████████| 234/234 [00:28<00:00,  8.22it/s]


[Epoch 99] Train Loss: 0.0280 Accuracy : 99.04%
Validation Accuracy: 86.62%


Epoch 100/100: 100%|██████████| 234/234 [00:28<00:00,  8.11it/s]


[Epoch 100] Train Loss: 0.0320 Accuracy : 99.01%
Validation Accuracy: 86.75%

📊 Test Set Evaluation (Best Model 기준):
✅ Test Accuracy: 9.27%
              precision    recall  f1-score   support

         0.0     0.8172    0.7964    0.8066       275
         1.0     0.8947    0.9067    0.9007       525

    accuracy                         0.8688       800
   macro avg     0.8560    0.8515    0.8536       800
weighted avg     0.8681    0.8688    0.8683       800

AUC: 0.9176
Confusion Matrix:
[[219  56]
 [ 49 476]]


In [ ]:
import os
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import torch.nn.functional as F
from tqdm import tqdm
import gc


# --- 설정 ---
SLICE_ROOT = "/data1/lidc-idri/slices"
BATCH_SIZE = 16
NUM_EPOCHS = 100
LR = 1e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- 라벨 추출 함수 ---
def extract_label_from_filename(filename):
    try:
        score = int(filename.split("_")[-1].replace(".npy", ""))
        if score == 3:
            return None
        return 1 if score >= 4 else 0
    except:
        return None

# --- 파일 리스트 구성 ---
all_files = glob(os.path.join(SLICE_ROOT, "LIDC-IDRI-*", "*.npy"))
file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]
files, labels = zip(*file_label_pairs)

# --- 3-way split (70% train / 15% val / 15% test) ---
train_files, temp_files, train_labels, temp_labels = train_test_split(
    files, labels, test_size=0.3, random_state=42
)
val_files, test_files, val_labels, test_labels = train_test_split(
    temp_files, temp_labels, test_size=0.5, random_state=42
)

# --- Transform 정의 ---
common_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),  # 모든 입력을 동일 크기로 맞춤
    transforms.ToTensor()
])

train_transform = common_transform
val_transform = common_transform

# --- Dataset 정의 ---
class LIDCDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, index):
        file_path = self.file_paths[index]
        label = self.labels[index]

        img = np.load(file_path).astype(np.float32)
        img = np.clip(img, -1000, 400)
        img = (img + 1000) / 1400.0  # normalize to 0~1
        img = np.stack([img] * 3, axis=-1)  # [H, W, 3] for RGB PIL input

        if self.transform:
            img_tensor = self.transform(img)  # transform includes ToTensor
        else:
            img_tensor = torch.tensor(img).permute(2, 0, 1).float()

        return img_tensor, torch.tensor(label).float()

# --- DataLoader 정의 ---
train_dataset = LIDCDataset(train_files, train_labels, transform=train_transform)
val_dataset = LIDCDataset(val_files, val_labels, transform=val_transform)
test_dataset = LIDCDataset(test_files, test_labels, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# --- 모델 정의 ---
model = models.resnet18(pretrained=True)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.fc = nn.Linear(model.fc.in_features, 1)
model = model.to(DEVICE)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

# --- 저장 경로 및 변수 초기화 ---
save_dir = os.path.join(os.path.dirname(os.getcwd()), "pth")
os.makedirs(save_dir, exist_ok=True)
best_val_acc = 0.0

# --- 학습 루프 ---
for epoch in range(NUM_EPOCHS):
    model.train()

    epoch_loss = 0
    correct = 0
    total = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}"):
        images = images.to(DEVICE)
        labels = labels.unsqueeze(1).to(DEVICE)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        predicted = torch.sigmoid(outputs) >= 0.5
        total += labels.size(0)
        correct += (predicted == labels.long()).sum().item()

        epoch_loss += loss.item()
        train_acc = correct / total

    print(f"[Epoch {epoch+1}] Train Loss: {epoch_loss/len(train_loader):.4f} Accuracy : {train_acc * 100:.2f}%")

    gc.collect()
    torch.cuda.empty_cache()

    # --- Validation ---
    model.eval()
    correct = total = 0
    y_true = []
    y_pred = []

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)

            outputs = model(images)
            preds = (torch.sigmoid(outputs) > 0.5).squeeze().long()

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

            correct += (preds == labels.long()).sum().item()
            total += labels.size(0)

    val_acc = correct / total
    print(f"Validation Accuracy: {val_acc * 100:.2f}%")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), os.path.join(save_dir, "best_model_resnet18.pth"))
        print("✅ Best model saved!")


# --- Test (최종 평가) ---
print("\n\U0001F4CA Test Set Evaluation (Best Model \uAE30\uc900):")
model.load_state_dict(torch.load(os.path.join(save_dir, "best_model_resnet18.pth")))
model.eval()

correct = total = 0
y_true = []
y_pred = []
y_probs = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(DEVICE)
        labels = labels.unsqueeze(1).float().to(DEVICE)

        outputs = model(images)
        probs = torch.sigmoid(outputs).squeeze()
        preds = (probs > 0.5).long()

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())
        y_probs.extend(probs.cpu().numpy())

        correct += (preds == labels.long()).sum().item()
        total += labels.size(0)

test_acc = correct / total
print(f"\u2705 Test Accuracy: {test_acc:.2f}%")
print(classification_report(y_true, y_pred, digits=4))

# AUC 출력
try:
    auc_score = roc_auc_score(y_true, y_probs)
    print(f"AUC: {auc_score:.4f}")
except ValueError:
    print("AUC \uacc4\uc0b0 \uc2e4\ud328: \uc591/\uc74c \ud074\ub798\uc2a4\uac00 \ubaa8\ub450 \uc788\uc5b4\uc57c \ud568.")

# Confusion Matrix 출력
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)
